In [1]:
!pip install geemap --upgrade

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: C:\Users\Easy Services Pro\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
# Installation de l'API Google Earth Engine
!pip install earthengine-api --upgrade

# Installation de la bibliothèque de visualisation interactive
!pip install geemap

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: C:\Users\Easy Services Pro\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: C:\Users\Easy Services Pro\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [3]:
import ee

# Lance la procédure d'authentification
ee.Authenticate()

True

In [4]:
# -------------------------------------------------------------------------
# CONFIGURATION DU PROJET
# -------------------------------------------------------------------------
NOM_DU_PROJET = 'math-soce-project1'

# Initialisation de la connexion GEE
ee.Initialize(project=NOM_DU_PROJET)

# Importation de geemap pour la carte interactive
import geemap

In [11]:
import ee
import folium
import os
import pandas as pd
import numpy as np

# Créer le dossier output s'il n'existe pas
output_dir = "output"
os.makedirs(output_dir, exist_ok=True)

# Données
img = ee.ImageCollection('ECMWF/ERA5/DAILY').filterDate('2015-01-01', '2020-12-01');
ghana = ee.FeatureCollection('FAO/GAUL/2015/level0').filter(ee.Filter.eq('ADM0_NAME', 'Ghana'))

print(ghana)
print(img)

ghana = img.filter(ee.Filter.eq('ADM1_NAME',
'Ghana'));
print(ghana)

ee.FeatureCollection({
  "functionInvocationValue": {
    "functionName": "Collection.filter",
    "arguments": {
      "collection": {
        "functionInvocationValue": {
          "functionName": "Collection.loadTable",
          "arguments": {
            "tableId": {
              "constantValue": "FAO/GAUL/2015/level0"
            }
          }
        }
      },
      "filter": {
        "functionInvocationValue": {
          "functionName": "Filter.equals",
          "arguments": {
            "leftField": {
              "constantValue": "ADM0_NAME"
            },
            "rightValue": {
              "constantValue": "Ghana"
            }
          }
        }
      }
    }
  }
})
ee.ImageCollection({
  "functionInvocationValue": {
    "functionName": "Collection.filter",
    "arguments": {
      "collection": {
        "functionInvocationValue": {
          "functionName": "ImageCollection.load",
          "arguments": {
            "id": {
              "constantValue":

In [ ]:
img = ee.FeatureCollection('FAO/GAUL/2015/level0')
.filter(ee.Filter.eq('ADM0_NAME', 'Ghana'));
climat = ee.ImageCollection('ECMWF/ERA5_LAND/MONTHLY_AGGR')
 .select('temperature_2m', 'total_precipitation_sum')
 .filter(ee.Filter.calendarRange(1990, 2020, 'year'))
 .filter(ee.Filter.calendarRange(4, 5, 'month')); 


In [12]:
img = ee.ImageCollection('ECMWF/ERA5/DAILY')
 .filterBounds(ghana)
 .filter(ee.Filter.date('2015-01-01', '2020-12-01'));
print(img)
print(img.size())

IndentationError: unexpected indent (909398417.py, line 2)

In [ ]:
# =========================================================================
# ANALYSE DE L'ÉTALEMENT URBAIN EN TANZANIE (2017-2022)
# Version optimisée avec calculs régionaux uniquement
# =========================================================================

import ee
import pandas as pd
import numpy as np

# Période d'analyse
ANNEE_DEBUT = 2015
ANNEE_FIN = 2020
T = ANNEE_FIN - ANNEE_DEBUT  # 5 ans

# Assets des limites administratives
GADM_L1_ASSET_ID = 'projects/math-soce-project1/assets/GADM_Tanzanie_ADMIN1'
REGION_COLONNE = 'NAME_1'
GADM_L0_ASSET_ID = 'projects/math-soce-project1/assets/GADM_Tanzanie_ADMIN0'

# Assets d'occupation du sol (6 tuiles par année)
OCCUPATION_2017_ASSETS = [
    'projects/travaux-pratique-478314/assets/Occupation_2017_1',
    'projects/travaux-pratique-478314/assets/Occupation_2017_2',
    'projects/travaux-pratique-478314/assets/Occupation_2017_3',
    'projects/travaux-pratique-478314/assets/Occupation_2017_4',
    'projects/travaux-pratique-478314/assets/Occupation_2017_5',
    'projects/travaux-pratique-478314/assets/Occupation_2017_6'
]

OCCUPATION_2022_ASSETS = [
    'projects/travaux-pratique-478314/assets/Occupation_2022_1',
    'projects/travaux-pratique-478314/assets/Occupation_2022_2',
    'projects/travaux-pratique-478314/assets/Occupation_2022_3',
    'projects/travaux-pratique-478314/assets/Occupation_2022_4',
    'projects/travaux-pratique-478314/assets/Occupation_2022_5',
    'projects/travaux-pratique-478314/assets/Occupation_2022_6'
]

# Assets de population WorldPop
POPULATION_2017_ASSET = 'projects/travaux-pratique-478314/assets/worldpop_Tanzanie_2017'
POPULATION_2022_ASSET = 'projects/travaux-pratique-478314/assets/worldpop_Tanzanie_2022'

# -------------------------------------------------------------------------
# PARAMÈTRES D'ANALYSE
# -------------------------------------------------------------------------

# Classe représentant les zones bâties dans vos rasters d'occupation
CLASSE_BATIE = 7

# Résolution spatiale des données d'occupation du sol (en mètres)
# D'après la vérification : 10m × 10m par pixel
RESOLUTION_OCCUPATION_M = 10
SUPERFICIE_PIXEL_KM2 = (RESOLUTION_OCCUPATION_M ** 2) / 1e6  # = 0.0001 km²

# Échelles de calcul
SCALE_OCCUPATION = 10   # Résolution native des données d'occupation (10m)
SCALE_POPULATION = 100  # Résolution WorldPop (100m)

# Projection UTM pour la Tanzanie
TARGET_CRS = 'EPSG:32735'  # UTM Zone 35S

#FACTEUR_CORRECTION_POP = 1/100  # Diviser par 100

# -------------------------------------------------------------------------
# INITIALISATION DE GOOGLE EARTH ENGINE
# -------------------------------------------------------------------------
ee.Initialize(project=NOM_DU_PROJET)

print("="*80)
print("     ANALYSE DE L'ÉTALEMENT URBAIN - TANZANIE")
print(f"     Période : {ANNEE_DEBUT} - {ANNEE_FIN}")
print("="*80)
print(f"\nParamètres de l'analyse :")
print(f"  • Résolution occupation du sol : {RESOLUTION_OCCUPATION_M}m")
print(f"  • Superficie par pixel : {SUPERFICIE_PIXEL_KM2} km²")
print(f"  • Classe bâtie : {CLASSE_BATIE}")
#print(f"  • Facteur correction population : {FACTEUR_CORRECTION_POP}")
print("="*80 + "\n")

# =========================================================================
# ÉTAPE 1 : CHARGEMENT DES LIMITES ADMINISTRATIVES
# =========================================================================

print("ÉTAPE 1 : Chargement des limites administratives")
print("-" * 80)

# Charger les régions (niveau 1)
admin_regions = ee.FeatureCollection(GADM_L1_ASSET_ID)

# Charger la géométrie nationale (niveau 0)
aoi_national = ee.FeatureCollection(GADM_L0_ASSET_ID).first().geometry()

# Obtenir le nombre de régions
num_regions = admin_regions.size().getInfo()

print(f"✓ Nombre de régions (niveau 1) : {num_regions}")
print(f"✓ Géométrie nationale chargée")
print(f"✓ Projection cible : {TARGET_CRS}\n")

# =========================================================================
# ÉTAPE 2 : PRÉPARATION DES IMAGES D'OCCUPATION DU SOL
# =========================================================================

print("ÉTAPE 2 : Préparation des mosaïques d'occupation du sol")
print("-" * 80)

def create_occupation_mosaic(asset_list, year):
    """
    Crée une mosaïque à partir des tuiles d'occupation du sol.

    Args:
        asset_list: Liste des IDs des assets à mosaïquer
        year: Année des données (pour le nommage)

    Returns:
        Image EE de la mosaïque clippée à l'AOI nationale
    """
    images = []

    # Charger chaque tuile
    for i, asset_id in enumerate(asset_list, 1):
        try:
            img = ee.Image(asset_id).select(['b1'])
            images.append(img)
            print(f"  ✓ Tuile {i}/6 chargée")
        except Exception as e:
            print(f"  ✗ Erreur tuile {i}/6 : {str(e)[:50]}")

    if len(images) == 0:
        raise Exception(f"Aucune image chargée pour {year}")

    # Créer la mosaïque
    mosaic = ee.ImageCollection(images).mosaic()

    # Clipper à l'AOI nationale
    mosaic_clipped = mosaic.clip(aoi_national)

    print(f"  → Mosaïque {year} créée ({len(images)}/6 tuiles)")

    return mosaic_clipped

# Créer les mosaïques pour 2017 et 2022
print(f"\nAnnée {ANNEE_DEBUT} :")
occupation_2017 = create_occupation_mosaic(OCCUPATION_2017_ASSETS, ANNEE_DEBUT)

print(f"\nAnnée {ANNEE_FIN} :")
occupation_2022 = create_occupation_mosaic(OCCUPATION_2022_ASSETS, ANNEE_FIN)

print("\n✓ Mosaïques d'occupation créées\n")

# =========================================================================
# ÉTAPE 3 : EXTRACTION DES ZONES BÂTIES
# =========================================================================

print("ÉTAPE 3 : Extraction des zones bâties")
print("-" * 80)

# Avant d'extraire, inspectons la première image pour connaître les valeurs
print("\nInspection des valeurs de classes d'occupation du sol...")
sample_values_2017 = occupation_2017.sample(
    region=aoi_national,
    scale=100,
    numPixels=100,
    geometries=False
).aggregate_array('b1').distinct().sort().getInfo()

print(f"Valeurs uniques trouvées dans les données 2017 : {sample_values_2017}")

def extract_built_area(occupation_image, year):
    """
    Extrait les pixels de la classe bâtie et calcule leur superficie.

    Args:
        occupation_image: Image d'occupation du sol
        year: Année (pour le nommage des bandes)

    Returns:
        Image où chaque pixel bâti = superficie en km²
    """
    # Créer un masque binaire : 1 = bâti, 0 = autre
    built_mask = occupation_image.eq(CLASSE_BATIE)

    # Appliquer le masque (ne garder que les pixels bâtis)
    # Chaque pixel bâti représente SUPERFICIE_PIXEL_KM2 km²
    #built_area = built_mask.selfMask().multiply(SUPERFICIE_PIXEL_KM2)
    built_area = built_mask.multiply(SUPERFICIE_PIXEL_KM2)

    # Renommer la bande
    built_area_named = built_area.rename(f'V_{year}')

    return built_area_named

# Extraire les zones bâties pour les deux années
V_2017 = extract_built_area(occupation_2017, ANNEE_DEBUT)
V_2022 = extract_built_area(occupation_2022, ANNEE_FIN)

print(f"✓ Zones bâties extraites (classe {CLASSE_BATIE})")
print(f"  • Chaque pixel bâti = {SUPERFICIE_PIXEL_KM2} km²\n")

# =========================================================================
# ÉTAPE 4 : PRÉPARATION DES DONNÉES DE POPULATION
# =========================================================================

print("ÉTAPE 4 : Préparation des données de population")
print("-" * 80)

def prepare_population(asset_id, year):
    """
    Charge et prépare les données de population avec correction du facteur.

    WorldPop fournit le nombre d'habitants par pixel de 100m × 100m.
    Vos données semblent être multipliées par 100, donc on applique
    le facteur de correction.

    Args:
        asset_id: ID de l'asset WorldPop
        year: Année (pour le nommage)

    Returns:
        Image de population corrigée
    """
    # Charger l'image
    pop_raw = ee.Image(asset_id).select(['b1'])

    # Appliquer le facteur de correction (diviser par 100)
    #pop_corrected = pop_raw.multiply(FACTEUR_CORRECTION_POP)

    # Clipper et renommer
    pop_final = pop_raw.clip(aoi_national).rename(f'Pop_{year}')

    return pop_final

# Préparer les images de population pour 2017 et 2022
Pop_2017 = prepare_population(POPULATION_2017_ASSET, ANNEE_DEBUT)
Pop_2022 = prepare_population(POPULATION_2022_ASSET, ANNEE_FIN)

print(f"✓ Données de population préparées")
#print(f"  • Facteur de correction appliqué : {FACTEUR_CORRECTION_POP}")
print(f"  • Les valeurs sont maintenant en habitants par pixel (1km)\n")

In [3]:
import ee
import folium
import os

# Initialisation EE
ee.Initialize()

# Créer le dossier output s'il n'existe pas
output_dir = "output"
os.makedirs(output_dir, exist_ok=True)

# Données
total_precipitation = ee.FeatureCollection('ECMWF/ERA5/DAILY')
ghana = ee.FeatureCollection('FAO/GAUL/2015/level0').filter(ee.Filter.eq('ADM0_NAME', 'Ghana'))
i = total_precipitation.clip(ghana)



EEException: Please authorize access to your Earth Engine account by running

earthengine authenticate

in your command line, or ee.Authenticate() in Python, and then retry.